In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Define image preprocessing function
def preprocess_image(image):
    image = tf.image.resize(image, (224, 224))  # Resize image to match model input size
    image = tf.keras.applications.resnet.preprocess_input(image)  # Preprocess image according to ResNet requirements
    return image


In [3]:
# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)

In [4]:
# Freeze convolutional layers to prevent retraining
for layer in base_model.layers:
    layer.trainable = False

In [5]:
# Add custom classification head for multi-class classification
x = GlobalAveragePooling2D()(base_model.output)
output = Dense(4, activation='softmax')(x)  # Multi-class classification (4 classes: non-toxic, nudity, gore, hate)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

In [6]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# Image data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_image,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [8]:
# Load and preprocess dataset for multi-class classification
train_generator = train_datagen.flow_from_directory(
    'train_data_directory',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  # Update class mode for multi-class classification
    shuffle=False,  # Ensure consistent ordering of images and labels
    classes=['non_toxic', 'nudity_and_pornography', 'gore_and_violence', 'hate_symbols_and_actions']  # Specify class labels
)

Found 1201 images belonging to 4 classes.


In [11]:
# Train the model
model.fit(train_generator, epochs=10)

Epoch 1/10


UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000002620229AF20>
Traceback (most recent call last):

  File "C:\Users\TARUN\anaconda3\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "C:\Users\TARUN\anaconda3\Lib\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\TARUN\anaconda3\Lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\TARUN\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py", line 250, in _get_iterator
    for i, batch i

## Input image for inference
input_image = tf.keras.preprocessing.image.load_img('input_image.jpg', target_size=(224, 224))
input_image = tf.keras.preprocessing.image.img_to_array(input_image)
input_image = preprocess_image(input_image)
input_image = tf.expand_dims(input_image, axis=0)

In [27]:
# Make prediction for multi-class classification
prediction = model.predict(input_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step


In [28]:
# Define thresholds for each category
thresholds = {
    'non_toxic': 0.1,
    'nudity_and_pornography': 0.3,
    'gore_and_violence': 0.3,
    'hate_symbols_and_actions': 0.30
}


In [29]:
# Class labels
class_labels = ['non_toxic', 'nudity_and_pornography', 'gore_and_violence', 'hate_symbols_and_actions']

In [30]:
# Determine toxicity categories based on thresholds
toxic_categories = [(label, prob) for label, prob in zip(class_labels, prediction[0]) if prob >= thresholds.get(label, 0)]

In [31]:
# Sort toxic categories by probability in descending order
toxic_categories.sort(key=lambda x: x[1], reverse=True)

In [32]:
# Get the highest toxic category
if toxic_categories:
    highest_toxic_category = toxic_categories[0]
    print("Predicted Highest Toxic Category:", highest_toxic_category)
else:
    print("Predicted Non-Toxic Image")

Predicted Toxicity Categories: [('non_toxic', 0.47266367)]


In [17]:
# Continuously learn from the input image (adjust learning rate as needed)
learning_rate = 0.001  # Adjust learning rate based on your requirements
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(input_image, prediction)  # Update model with predicted probability

1/1 ━━━━━━━━━━━━━━━━━━━━ 24s 24s/step - accuracy: 1.0000 - loss: 0.8814


In [18]:
# Evaluate the model on the test dataset
test_generator = train_datagen.flow_from_directory(
    'test_data_directory',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False,
    classes=['non_toxic', 'nudity_and_pornography', 'gore_and_violence', 'hate_symbols_and_actions']
)

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Found 18 images belonging to 4 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - accuracy: 0.8889 - loss: 0.2498
Test Loss: 0.2498454451560974
Test Accuracy: 0.8888888955116272
